In [1]:
!pip install pandas polars numpy treys

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import polars as pl
import random

In [3]:
import random
import pandas as pd
from treys import Card, Evaluator

def get_random_hand(deck):
    return [deck.pop(), deck.pop()]

def get_random_flop(deck):
    return [deck.pop(), deck.pop(), deck.pop()]

def simulate_equity_vs_5(player_hand, flop, num_simulations=500):
    evaluator = Evaluator()
    win_count = 0

    all_used = set(player_hand + flop)
    full_deck = set(Card.new(rank + suit) for rank in '23456789TJQKA' for suit in 'shdc')

    for _ in range(num_simulations):
        deck = list(full_deck - all_used)
        random.shuffle(deck)

        opponents = []
        used = set()
        while len(opponents) < 5:
            h1, h2 = deck.pop(), deck.pop()
            if h1 not in used and h2 not in used:
                opponents.append([h1, h2])
                used.add(h1)
                used.add(h2)

        turn, river = deck.pop(), deck.pop()
        board = flop + [turn, river]

        all_hands = [player_hand] + opponents
        scores = [evaluator.evaluate(board, hand) for hand in all_hands]
        best_score = min(scores)
        winners = [i for i, score in enumerate(scores) if score == best_score]

        if 0 in winners:
            win_count += 1 / len(winners)

    return round(100 * win_count / num_simulations, 2)

def card_to_str(card):
    return Card.int_to_str(card)

problems = []
for _ in range(500):
    ranks = '23456789TJQKA'
    suits = 'shdc'
    full_deck = [Card.new(r + s) for r in ranks for s in suits]
    random.shuffle(full_deck)

    player_hand = get_random_hand(full_deck)
    flop = get_random_flop(full_deck)
    equity = simulate_equity_vs_5(player_hand, flop, num_simulations=500)

    problems.append({
        "PlayerCard1": card_to_str(player_hand[0]),
        "PlayerCard2": card_to_str(player_hand[1]),
        "FlopCard1": card_to_str(flop[0]),
        "FlopCard2": card_to_str(flop[1]),
        "FlopCard3": card_to_str(flop[2]),
        "EquityPercent": equity
    })

df = pd.DataFrame(problems)
df.to_csv("poker_equity_problems.csv", index=False)
print("Saved to poker_equity_problems.csv")


KeyboardInterrupt: 

In [32]:
df = pl.read_csv('poker_equity_problems.csv')

In [42]:
row_num = random.randrange(0,500)

In [45]:
df.row(row_num)[0]

'Tc.png'

In [46]:
"asdf" + "123"

'asdf123'

In [5]:
import random
from treys import Card, Deck, Evaluator

def compute_equity(player_hand, flop_cards, num_simulations, num_opponents):
    """
    Compute the equity of a hand using Monte Carlo simulation with treys.
    
    Parameters:
    - player_hand: list of 2 card strings, e.g., ['As', 'Kd']
    - flop_cards: list of 3 card strings, e.g., ['2h', '7c', 'Td']
    - num_simulations: number of simulations to run
    - num_opponents: number of opponents
    
    Returns:
    - equity (float between 0 and 1)
    """
    evaluator = Evaluator()
    player_hand_int = [Card.new(card) for card in player_hand]
    flop_int = [Card.new(card) for card in flop_cards]

    wins = 0

    for _ in range(num_simulations):
        # Create and shuffle a fresh deck
        deck = Deck()
        
        # Remove known cards
        for card in player_hand_int + flop_int:
            deck.cards.remove(card)

        # Deal remaining board (2 more cards)
        remaining_board = deck.draw(2)
        board = flop_int + remaining_board

        # Deal opponent hands
        opponent_hands = [deck.draw(2) for _ in range(num_opponents)]

        # Evaluate hands
        player_score = evaluator.evaluate(board, player_hand_int)
        best_opponent_score = min(evaluator.evaluate(board, opp_hand) for opp_hand in opponent_hands)

        if player_score < best_opponent_score:
            wins += 1
        elif player_score == best_opponent_score:
            wins += 0.5  # account for tie

    equity = round((wins / num_simulations) * 100)
    return equity


In [74]:
compute_equity(['3s','As'],['Ts','2s','Ad'],1000,5)

64

In [55]:
## Extract data
df = pl.read_csv('poker_equity_problems.csv')

playercard1 = df.row(5)[0]
playercard2 = df.row(5)[1]
flop1 = df.row(5)[2]
flop2 = df.row(5)[3]
flop3 = df.row(5)[4]

In [57]:
playercard1[:-4]

'Ad'

In [73]:
compute_equity([playercard1[:-4], playercard2[:-4]],
               [flop1[:-4], flop2[:-4], flop3[:-4]],
               1000,5
               )

21

In [9]:
from treys import Card, Evaluator, Deck
from itertools import combinations

def calculate_equity(my_hand, opponent_hand, flop):
    """
    Calculate win probability in heads-up Texas Hold'em given hands and flop.
    
    Args:
        my_hand: List of 2 card strings (e.g., ['As', 'Kh'])
        opponent_hand: List of 2 card strings (e.g., ['Qd', '9c'])
        flop: List of 3 card strings (e.g., ['Jh', 'Ts', '2d'])
    
    Returns:
        dict: Contains win_rate, tie_rate, lose_rate, and total_simulations
    """
    evaluator = Evaluator()
    
    # Convert card strings to Card objects
    my_cards = [Card.new(card) for card in my_hand]
    opponent_cards = [Card.new(card) for card in opponent_hand]
    flop_cards = [Card.new(card) for card in flop]
    
    # Create deck and remove known cards
    deck = Deck()
    known_cards = my_cards + opponent_cards + flop_cards
    remaining_cards = [card for card in deck.cards if card not in known_cards]
    
    wins = 0
    ties = 0
    losses = 0
    total_simulations = 0
    
    # Generate all possible turn and river combinations
    for turn_river in combinations(remaining_cards, 2):
        turn, river = turn_river
        board = flop_cards + [turn, river]
        
        # Evaluate both hands
        my_score = evaluator.evaluate(board, my_cards)
        opponent_score = evaluator.evaluate(board, opponent_cards)
        
        # Lower score is better in treys
        if my_score < opponent_score:
            wins += 1
        elif my_score == opponent_score:
            ties += 1
        else:
            losses += 1
        
        total_simulations += 1
    
    # Calculate proportions
    win_rate = wins / total_simulations
    tie_rate = ties / total_simulations
    lose_rate = losses / total_simulations
    
    return {
        'win_rate': win_rate,
        'tie_rate': tie_rate,
        'lose_rate': lose_rate,
        'wins': wins,
        'ties': ties,
        'losses': losses,
        'total_simulations': total_simulations
    }

def print_equity_results(results):
    """Helper function to nicely print the equity results"""
    print(f"Total simulations: {results['total_simulations']}")
    print(f"Wins: {results['wins']} ({results['win_rate']:.2%})")
    print(f"Ties: {results['ties']} ({results['tie_rate']:.2%})")
    print(f"Losses: {results['losses']} ({results['lose_rate']:.2%})")
    print(f"Equity: {results['win_rate'] + results['tie_rate']/2:.2%}")

# Example usage
if __name__ == "__main__":
    # Example: Pocket Aces vs Pocket Kings on a rainbow flop
    my_hand = ['Ad', 'Kd']
    opponent_hand = ['Ac', 'Qc'] 
    flop = ['As', '6c', '2h']
    
    results = calculate_equity(my_hand, opponent_hand, flop)
    print_equity_results(results)
    
    print("\n" + "="*50 + "\n")


Total simulations: 990
Wins: 813 (82.12%)
Ties: 12 (1.21%)
Losses: 165 (16.67%)
Equity: 82.73%


